<a href="https://colab.research.google.com/github/Konoko2004/TheProject/blob/main/yFinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Install related libraries

!pip install yfinance

!pip install dash
import pandas as pd
import yfinance as yf

In [17]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Output, Input

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Testing')
    dcc.Graph(id = 'my-graph')
])

fig = px.bar(nio,x = 'Ticker',y = 'Volume')


if __name__ == '__main__':
  app.run_server()

SyntaxError: ignored

In [7]:
df1 = pd.DataFrame([[1,2,3,4]],
                   columns = ['a','b','c','d'])
df2 = pd.DataFrame([[1,2,3,4,5],[1,4,9,16,25],[1,16,87,12,41]],
                   columns = ['a','b','c','d','e'])

print(pd.concat([df1,df2],axis=1))

     a    b    c    d  a   b   c   d   e
0  1.0  2.0  3.0  4.0  1   2   3   4   5
1  NaN  NaN  NaN  NaN  1   4   9  16  25
2  NaN  NaN  NaN  NaN  1  16  87  12  41


In [9]:
#Function to create a DF of a ticker. We will add them to a class of stocks later.

def create_Stock(tickerName):
    stock = yf.Ticker(tickerName)
    stockhistory = stock.history(period = '1y')
    stock_df = pd.DataFrame(stockhistory)
    stock_df['Ticker'] = tickerName
    stock_df.reset_index(inplace = True) # push date index to become a column.
    stock_df['Date'] = stock_df['Date'].dt.strftime('%d/%m/%Y')  #dd/mm/yyyy format.
    return stock_df

In [ ]:
#Function to calculate 

In [ ]:
class WatchList:
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.buy

    

In [10]:
nio = create_Stock("nio")
nio


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
0,28/05/2020,3.980000,4.120000,3.750000,3.830000,62756900,0,0,nio
1,29/05/2020,3.820000,4.000000,3.730000,3.980000,50303000,0,0,nio
2,01/06/2020,4.000000,4.330000,3.960000,4.260000,69222000,0,0,nio
3,02/06/2020,4.360000,4.730000,4.350000,4.700000,82476600,0,0,nio
4,03/06/2020,5.000000,5.700000,4.980000,5.600000,199210000,0,0,nio
...,...,...,...,...,...,...,...,...,...
249,24/05/2021,34.259998,36.490002,33.560001,35.889999,67244400,0,0,nio
250,25/05/2021,36.320000,37.639999,35.689999,36.070000,76628000,0,0,nio
251,26/05/2021,36.250000,38.080002,36.070000,37.759998,55929000,0,0,nio
252,27/05/2021,37.950001,38.980000,37.180000,38.910000,52933300,0,0,nio


In [ ]:
nio = yf.Ticker("nio")
niohistory = nio.history(period = '1y')
nio_df = pd.DataFrame(niohistory)
nio_df

nio_df['High-Low'] = nio_df['High']-nio_df['Low']
nio_df['Ticker'] = 
nio_df = nio_df[['Open','High','Low','Close','High-Low','Volume','Dividends','Stock Splits']]   #re-arrange columns

nio_df

,Open,High,Low,Close,High-Low,Volume,Dividends,Stock Splits
Date,,,,,,,,
2020-05-18,3.500000,3.690000,3.500000,3.610000,0.190000,27771300,0,0
2020-05-19,3.620000,3.750000,3.590000,3.690000,0.160000,21707400,0,0
2020-05-20,3.760000,3.820000,3.400000,3.480000,0.420000,52056900,0,0
2020-05-21,3.400000,3.440000,3.230000,3.330000,0.210000,41628400,0,0
2020-05-22,3.330000,3.340000,3.180000,3.270000,0.160000,22887300,0,0
...,...,...,...,...,...,...,...,...
2021-05-12,34.380001,35.130001,33.480000,33.680000,1.650002,75993700,0,0
2021-05-13,33.849998,34.270000,30.709999,31.219999,3.560001,113686700,0,0
2021-05-14,31.940001,33.689999,31.559999,33.419998,2.129999,80032000,0,0


In [ ]:
def moving_avg(dataframe, period = 20,Moving_avg = ''):

    self

In [ ]:
nio = create_Stock(tickerName = 'nio')

nio['High-Low'] = nio['High']-nio['Low']
nio

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,High-Low
0,26/05/2020,3.420000,3.830000,3.400000,3.820000,59847900,0,0,nio,0.430000
1,27/05/2020,4.010000,4.200000,3.900000,4.170000,70718900,0,0,nio,0.300000
2,28/05/2020,3.980000,4.120000,3.750000,3.830000,62756900,0,0,nio,0.370000
3,29/05/2020,3.820000,4.000000,3.730000,3.980000,50303000,0,0,nio,0.270000
4,01/06/2020,4.000000,4.330000,3.960000,4.260000,69222000,0,0,nio,0.370000
...,...,...,...,...,...,...,...,...,...,...
249,20/05/2021,33.980000,34.509998,33.639999,34.330002,48991700,0,0,nio,0.869999
250,21/05/2021,34.660000,34.779999,33.869999,34.060001,49419300,0,0,nio,0.910000
251,24/05/2021,34.259998,36.490002,33.560001,35.889999,67244400,0,0,nio,2.930000
252,25/05/2021,36.320000,37.639999,35.689999,36.070000,76545800,0,0,nio,1.950001


In [ ]:
nio_tidied = pd.melt(nio,id_vars=['Volume','Dividends','Stock Splits','Date','Ticker'],
                     var_name='Position',value_name = 'Price')

nio_tidied

,Volume,Dividends,Stock Splits,Date,Ticker,Position,Price
0,59847900,0,0,26/05/2020,nio,Open,3.420000
1,70718900,0,0,27/05/2020,nio,Open,4.010000
2,62756900,0,0,28/05/2020,nio,Open,3.980000
3,50303000,0,0,29/05/2020,nio,Open,3.820000
4,69222000,0,0,01/06/2020,nio,Open,4.000000
...,...,...,...,...,...,...,...
1265,48991700,0,0,20/05/2021,nio,High-Low,0.869999
1266,49419300,0,0,21/05/2021,nio,High-Low,0.910000
1267,67244400,0,0,24/05/2021,nio,High-Low,2.930000
1268,76545800,0,0,25/05/2021,nio,High-Low,1.950001
